In [1]:
import pandas as pd
import numpy as np
from datetime import date

# combined data - 2 tables

In [2]:
path_tables = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 2 tables/"

In [3]:
filename_careers = "careers.xlsx"
careers = pd.read_excel(path_tables + filename_careers,dtype="str")
careers.shape

(9001, 12)

In [4]:
filename_orgtree = "orgtree.xlsx"
ot = pd.read_excel(path_tables + filename_orgtree,dtype="str")
ot.shape

(1711, 9)

In [5]:
filename_leadercareerlink = "leadercareerlink.xlsx"
lclink = pd.read_excel(path_tables + filename_leadercareerlink,dtype="str")
lclink.shape

(12617, 3)

In [6]:
# filename_leaderjoblink = "leaderjoblink.xlsx"

In [7]:
# filename_joborglink = "joborglink.xlsx"

# combined data - 3 queries

In [8]:
path_queries = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 3 queries/"

In [9]:
filename_leaderjobtransition = "leaderjobtransition.xlsx"
# ljt = pd.read_excel(path_queries + filename_leaderjobtransition,dtype="str")

In [10]:
today = date.today()
print(today)

2023-07-09


# functions

In [17]:
def merge_results(m):
    
    print("\nMerge Results...")
    print("")
    print("\tshape     :",m.shape)
    print("\tleft_only :",m[m["_merge"]=="left_only"].shape)
    print("\tboth      :",m[m["_merge"]=="both"].shape)
    print("\tright_only:",m[m["_merge"]=="right_only"].shape)

# 1. deprecate old version

In [11]:
table_name = "leaderjobtransition"
table_name

'leaderjobtransition'

In [12]:
filename_leaderjobtransition

'leaderjobtransition.xlsx'

In [13]:
filename_deprecated = table_name + "_deprecated_"+str(today)+".xlsx"
filename_deprecated

'leaderjobtransition_deprecated_2023-07-09.xlsx'

In [14]:
# deprecate old query
# oldtable = pd.read_excel(path_queries + filename_careers,dtype="str")
# oldtable.to_excel(path_queries + filename_deprecated,index=False)

# 2. refresh query using current tables

In [15]:
lclink.head(2)

,LeaderID,CareerString,CareerDateString_2022
0,리선권,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01
1,조경철,故 김정일 국가장의위원회 위원,2011.12


In [22]:
# get jobs from careers
jobs = careers[careers["IsJob"]=="True"]

In [29]:
# leader-job query - left join - includes IsJob True & False
key_columns = ["CareerString","CareerDateString_2022"]
lc = lclink.merge(jobs,on=key_columns,how="left",indicator=True)
lc.shape

(13607, 14)

In [30]:
merge_results(lc)


Merge Results...

	shape     : (13607, 14)
	left_only : (4237, 14)
	both      : (9370, 14)
	right_only: (0, 14)


In [32]:
# no "left-only" cases when IsJob=True
lc[(lc["IsJob"]=="True") & (lc["_merge"]=="left_only")]

,LeaderID,CareerString,CareerDateString_2022,IsJob,MultipleSubstrings,CareerStartYear,CareerStartMonth,CareerSubstring,OrgString,PrimaryInstitution,OrgName,Position,Notes,_merge


In [40]:
# leader-job query - inner join - only includes IsJob = True
key_columns = ["CareerString","CareerDateString_2022"]
lc = lclink.merge(jobs,on=key_columns,how="inner",indicator=False)
lc = lc.sort_values("LeaderID",ignore_index=True)
lc.shape

(9370, 13)

In [41]:
lc.head(10)

,LeaderID,CareerString,CareerDateString_2022,IsJob,MultipleSubstrings,CareerStartYear,CareerStartMonth,CareerSubstring,OrgString,PrimaryInstitution,OrgName,Position,Notes
0,강관주,조선노동당 대외연락부(구 사회문화부) 부장,1997.02,True,1,1997,2,NaN,조선노동당 대외연락부,노동당,당중앙위원회_대외연락부,부장,NaN
1,강관주,조선노동당 조직지도부 부부장,1986,True,1,1986,NaN,NaN,조선노동당 조직지도부,노동당,당중앙위원회_조직지도부,부부장,NaN
2,강관주,2010. 9 당 중앙위 후보위원,NaN,True,1,2010,9,NaN,노동당 중앙위원회,노동당,당중앙위원회,후보위원,NaN
3,강관주,조선노동당 중앙위원회 위원,1988.03,True,1,1988,3,NaN,조선노동당 중앙위원회,노동당,당중앙위원회,위원,NaN
4,강관주,1990. 8 조국평화통일위원회 부위원장,NaN,True,1,1990,8,NaN,조국평화통일위원회,내각,조국평화통일위원회,부위원장,NaN
5,강관주,1961. 체육과학연구소 연구원,NaN,True,1,1961,NaN,NaN,체육과학연구소,체육연구원,체육과학연구소,연구원,NaN
6,강관주,조선노동당 중앙위원회 후보위원,2010.09,True,1,2010,9,NaN,조선노동당 중앙위원회,노동당,당중앙위원회,후보위원,NaN
7,강관주,조선노동당 통일전선부 부장,1993.01,True,1,1993,1,NaN,조선노동당 통일전선부,노동당,당중앙위원회_통일전선부,부장,NaN
8,강관주,조선노동당 통일전선부 제1부부장,1989.03,True,1,1989,3,NaN,조선노동당 통일전선부 제1부,노동당,당중앙위원회_통일전선부,부장,NaN
9,강관주,조선노동당 통일전선부 조선인총연합회 담당 부부장,1988,True,1,1988,NaN,NaN,조선노동당 통일전선부,노동당,당중앙위원회_통일전선부,부부장,NaN


In [71]:
t = lc[(lc["LeaderID"]=="강관주") & (lc["CareerStartYear"].notna())]
t = t.sort_values(["CareerStartYear","CareerStartMonth"])
t = t.drop_duplicates(["CareerStartYear","PrimaryInstitution","OrgName"],keep="first")
t.shape

(22, 13)

In [106]:
# instantiate CareerStartDate
t["CareerStartDate"]=np.nan
t.loc[t["CareerStartMonth"].isna(),"CareerStartDate"] = t[t["CareerStartMonth"].isna()].apply(lambda x: x["CareerStartYear"] + "00",axis=1)
t.loc[(t["CareerStartMonth"].notna()) & (t.apply(lambda x: len(str(x["CareerStartMonth"])),axis=1)==2),"CareerStartDate"] = t[(t["CareerStartMonth"].notna()) & (t.apply(lambda x: len(str(x["CareerStartMonth"])),axis=1)==2)].apply(lambda x: x["CareerStartYear"] + x["CareerStartMonth"],axis=1)
t.loc[(t["CareerStartMonth"].notna()) & (t.apply(lambda x: len(str(x["CareerStartMonth"])),axis=1)==1),"CareerStartDate"] = t[(t["CareerStartMonth"].notna()) & (t.apply(lambda x: len(str(x["CareerStartMonth"])),axis=1)==1)].apply(lambda x: x["CareerStartYear"] + "0" + x["CareerStartMonth"],axis=1)

# 3. export new query

In [55]:
# refresh leaderjobtransition with new data
ljt_new.to_excel(path_tables + filename_careers,index=False)